In [1]:
import pandas as pd
from tqdm import tqdm_notebook as tqdm
df = pd.read_csv('preprocess_for_SQUAD.csv',index_col=0)
print(df.shape)
df.head(3)

(744, 21)


,59,45A,46A,47A,31D,44C,48,71D,78,50,...,LCNO,from,EXPNO,LCBK,CU1,SPEC,Baodian_predict,Y_label,string_Y_1,string_Y_2
3,"FORMOSA PLASTICS CORPORATION201,TUNG HWA N ROA...",MASS PVC RESIN B-57QUANTITY 175 MT AT 1300 USD...,1. SIGNED COMMERCIAL INVOICE IN 3 ORIGINALS AN...,1. ALL DOCUMENTS MUST BE IN ENGLISH.2. ALL DOC...,210321TAIWAN,210228.0,21/(REFER FIELD 47A CLAUSE NO.8),ALL BANK CHARGES OUTSIDE INDIA ARETO THE ACCOU...,+UPON RECEIPT OF CREDIT COMPLIANT DOCUMENTS AT...,OJUS PETROCHEMICALS LLPC 289 NIRALA NAGAR LUCK...,...,6281MLC00000321,017-202101-03.xlsx,11,"ICICI BANK LTD.,\nSHALIMAR TOWER, 31/54 M.G.MA...",OJUS PETROCHEMICALS LLPC 289 NIRALA\n NAGAR LU...,MASS PVC RESIN B-57,PVC RESIN B-57,MASS PVC RESIN B-57,0,19
4,NAN YA PLASTICS CORPORATION201 TUN HWA NORTH R...,PHTHALIC ANHYDRIDE PAQUANTITY 306 MT UNIT PRI...,1. SIGNED COMMERCIAL INVOICE IN ORIGINAL AND 4...,(A) ALL DOCUMENTS AND DRAFTS (IF CALLED FOR UN...,210221TAIWAN,NaN,021,NaN,IN REIMBURSEMENT OF NEGOTIATION MADE BY YOUIN ...,AHMED SAEED AFIFI FACTORY CO. FORRESINS LTD. P...,...,L072875,017-202101-03.xlsx,27,"NATIONAL COMMERCIAL BANK, THE\n(HEAD OFFICE)",AHMED SAEED AFIFI FACTORY CO. FORRE\nSINS LTD....,PHTHALIC ANHYDRIDE,PHTHALIC ANHYDRIDE PA,PHTHALIC ANHYDRIDE,0,18
28,"FORMOSA PLASTICS CORPORATION .201, TUNG HWA NO...",TERMS OF SALE CIF - NHAVA SHEVA PORT INDIAPVC ...,+1.CLEAN ON BOARD OCEAN BILLS OF LADING (FULL+...,+1.DOCUMENTS TO BE SENT DIRECTLY TO US IN ONE+...,210122TAIWAN,210107.0,NaN,ALL ADVISING BANK CHARGES ANDOTHER CHARGES OUT...,WE HEREBY UNDERTAKE WITH DRAWERS AND/OR BONAFI...,"NAVYUG CHEMICALS PVT LTD802, EMBASSY CHAMBER,3...",...,5524615920,017-202101-03.xlsx,11,CITIBANK N.A. (INTERNATIONAL\nBUSINESS AND FIN...,"NAVYUG CHEMICALS PVT LTD802, EMBASS\nY CHAMBER...",PVC RESIN S-65D,PVC RESIN,PVC RESIN S-65D,42,57


# 斷詞模型

In [2]:
import wordninja

寶典 = pd.read_excel('台塑企業_ 產品寶典20210303.xlsx')
產品集合 = list(set(寶典['品名'].values))
產品集合 = [str(i).lower() for i in 產品集合]
產品集合 = [ i for i in 產品集合 if len(i)>5]
print(產品集合[:5])

with open('wordninja_words.txt',encoding="utf-8") as f:
    wordninja_words_lst = f.read().split('\n')
print(wordninja_words_lst[:5])

SPEC集合 = df['SPEC'].values.tolist()
SPEC集合 = [str(i).lower() for i in SPEC集合]
print(SPEC集合[:5])

全部集合 = list(set(產品集合+SPEC集合+wordninja_words_lst))
print(全部集合[:5])

with open('全部集合.txt', 'w',encoding="utf-8") as f:
    lines = [i + '\n' for i in 全部集合]
    f.writelines(lines)
import gzip
import shutil
with open('全部集合.txt', 'rb') as f_in, gzip.open('全部集合.txt.gz', 'wb') as f_out:
    shutil.copyfileobj(f_in, f_out)
lm = wordninja.LanguageModel('全部集合.txt.gz')
' '.join(lm.split(df['45A'].values[0]))

['f24608-095a', 'pp yarn', 'polyester pique weftknitted piece dyed fabric', 'copolymer resin yungsox', 'el 128']
['the', 'of', 'in', 'a', 'and']
['mass pvc resin b-57', 'phthalic anhydride', 'pvc resin s-65d', 'tetrahydrofuran', 'pvc resin s-65d']
['urchins', '', 'moulin', 'stalking', 'typhus']


'MASS PVC RESIN B 57 QUANTITY 175 MT AT 1300 USD M TAS PER PROFORMA INVOICE NO A 0 B 002 11 DATED 10 12 2020 C IF PIPA VAV PORT INDIA IN COTE RMS 2010'

# 斷詞處理

In [3]:
lst = []
for i in tqdm(range(len(df))):
    lst.append(' '.join(lm.split(df['45A'].values[i])))
df['45A'] = lst
df[['45A','Y_label','string_Y_1','string_Y_2']].head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/744 [00:00<?, ?it/s]

,45A,Y_label,string_Y_1,string_Y_2
3,MASS PVC RESIN B 57 QUANTITY 175 MT AT 1300 US...,MASS PVC RESIN B-57,0,19
4,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,0,18
28,TERMS OF SALE C IF N HAVA SHEVA PORT INDIA PVC...,PVC RESIN S-65D,42,57
31,PRODUCT TETRAHYDROFURAN 998 PCT MIN QUANTITY 1...,TETRAHYDROFURAN,10,25
41,875 MTS OF PVC RESIN B 57 AND 360 MTS OF PVC R...,PVC RESIN S-65D,41,56


# 清洗

In [4]:
df = df[['45A','Y_label','string_Y_1','string_Y_2']]
df = df.dropna(axis=0)
df = df.reset_index(drop=True)
print(df.shape)
keep_lst = []
for i in range(len(df)):
    if df.iloc[i]['Y_label'] in df.iloc[i]['45A']:
        keep_lst.append(i)
df = df.iloc[keep_lst]
print(df.shape)

(609, 4)
(488, 4)


# 起始結束標註

In [5]:
def str2index(context,string):
    ys = context.find(string)
    ye = ys+len(string)
    return ys,ye

ys_lst = []
ye_lst = []
for i in range(len(df)):
    context= df['45A'].values[i]
    string = df['Y_label'].values[i]
    ys,ye = str2index(context,string)
    ys_lst.append(ys)
    ye_lst.append(ye)
df['string_Y_1'] = ys_lst
df['string_Y_2'] = ye_lst
df[['45A','Y_label','string_Y_1','string_Y_2']].head()

,45A,Y_label,string_Y_1,string_Y_2
1,PHTHALIC ANHYDRIDE PA QUANTITY 306 MT UNIT PRI...,PHTHALIC ANHYDRIDE,0,18
3,PRODUCT TETRAHYDROFURAN 998 PCT MIN QUANTITY 1...,TETRAHYDROFURAN,8,23
6,COMMODITY MONOETHYLENE GLYCOL MEG FIBER GRADE ...,MONOETHYLENE GLYCOL MEG FIBER GRADE,10,45
7,COMMODITY DIETHYLENE GLYCOL DEG QUANTITY 1000 ...,DIETHYLENE GLYCOL DEG,10,31
13,COMMODITY DIETHYLENE GLYCOL DEG QUANTITY 1000 ...,DIETHYLENE GLYCOL DEG,10,31


In [6]:
print(df.shape)

(488, 4)


# 保存

In [7]:
df.to_csv('preprocess_for_SQUAD_wordninja.csv')